In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import Window as WN
from pyspark.sql import types as TYPE

In [2]:
spark = SparkSession.builder.master("local[2]").appName("READ Operation").getOrCreate()
# spark = SparkSession.builder.master("local[2]").appName("DataSet").getOrCreate()

In [3]:
spark

In [4]:
filepath = "E:\workforce\spark\data\hudsonu-product\Customer.csv"

In [41]:
data = [
{"ChargingEvent" : 15554472,"CPID" : "AN07263",
"StartDate":"2017-10-29","StartTime":"13:30:00",
"EndDate":"2017-10-29","EndTime": "17:08:00",
"Energy":5.3,"PluginDuration":3.6333333333333333
},
    {"ChargingEvent" : 15329256,"CPID" : "AN15092",
"StartDate":"2017-10-14","StartTime":"17:37:00",
"EndDate":"2017-10-15","EndTime": "05:26:00",
"Energy":19.2,"PluginDuration":11.816666666666666
},
    {"ChargingEvent" : 2344473,"CPID" : "AN22594",
"StartDate":"2017-06-02","StartTime":"16:10:19",
"EndDate":"2017-06-03","EndTime": "13:03:21",
"Energy":11.5,"PluginDuration":20.88388888888889
},
    {"ChargingEvent" : 12184545,"CPID" : "AN10218",
"StartDate":"2017-03-20","StartTime":"21:43:37",
"EndDate":"2017-03-21","EndTime": "20:18:29",
"Energy":12.1,"PluginDuration":22.58111111111111
},
    {"ChargingEvent" : 11984777,"CPID" : "AN02137",
"StartDate":"2017-03-07","StartTime":"10:21:17",
"EndDate":"2017-03-08","EndTime": "18:10:15",
"Energy":7.8,"PluginDuration":31.816111111111113
}
]


data = [
    ("AN22594", 2344473, "03-06-2017", "13:03:21", 11.5, 20.88388889, "02-06-2017", "16:10:19"),
    ("AN10218", 12184545, "21-03-2017", "20:18:29", 12.1, 22.58111111, "20-03-2017", "21:43:37"),
    ("AN02137", 11984777, "08-03-2017", "18:10:15", 7.8, 31.81611111, "07-03-2017", "10:21:17"),
    ("AN07263", 15554472, "29-10-2017", "17:08:00", 5.3, 3.633333333, "29-10-2017", "13:30:00"),
    ("AN15092", 15329256, "15-10-2017", "05:26:00", 19.2, 11.81666667, "14-10-2017", "17:37:00")
]

columns = ["CPID", "ChargingEvent", "EndDate", "EndTime", "Energy", "PluginDuration", "StartDate", "StartTime"]

df = spark.createDataFrame(data, columns)
df.show()

+-------+-------------+----------+--------+------+--------------+----------+---------+
|   CPID|ChargingEvent|   EndDate| EndTime|Energy|PluginDuration| StartDate|StartTime|
+-------+-------------+----------+--------+------+--------------+----------+---------+
|AN22594|      2344473|03-06-2017|13:03:21|  11.5|   20.88388889|02-06-2017| 16:10:19|
|AN10218|     12184545|21-03-2017|20:18:29|  12.1|   22.58111111|20-03-2017| 21:43:37|
|AN02137|     11984777|08-03-2017|18:10:15|   7.8|   31.81611111|07-03-2017| 10:21:17|
|AN07263|     15554472|29-10-2017|17:08:00|   5.3|   3.633333333|29-10-2017| 13:30:00|
|AN15092|     15329256|15-10-2017|05:26:00|  19.2|   11.81666667|14-10-2017| 17:37:00|
+-------+-------------+----------+--------+------+--------------+----------+---------+



In [66]:
df = df.withColumn("PluginDuration",FS.round(FS.col("PluginDuration"),2))

In [67]:
df.show()

+-------+-------------+----------+--------+------+--------------+----------+---------+------------+
|   CPID|ChargingEvent|   EndDate| EndTime|Energy|PluginDuration| StartDate|StartTime|diff_in_hour|
+-------+-------------+----------+--------+------+--------------+----------+---------+------------+
|AN22594|      2344473|03-06-2017|13:03:21|  11.5|         20.88|02-06-2017| 16:10:19|       20.88|
|AN10218|     12184545|21-03-2017|20:18:29|  12.1|         22.58|20-03-2017| 21:43:37|       22.58|
|AN02137|     11984777|08-03-2017|18:10:15|   7.8|         31.82|07-03-2017| 10:21:17|       31.82|
|AN07263|     15554472|29-10-2017|17:08:00|   5.3|          3.63|29-10-2017| 13:30:00|        3.63|
|AN15092|     15329256|15-10-2017|05:26:00|  19.2|         11.82|14-10-2017| 17:37:00|       11.82|
+-------+-------------+----------+--------+------+--------------+----------+---------+------------+



In [69]:
max_duration = df.selectExpr("max(PluginDuration)").collect()[0][0]
avg_duration = df.selectExpr("avg(PluginDuration)").collect()[0][0]
data1 = [{"max_duration":max_duration,"avg_duration":avg_duration}]
data1

[{'max_duration': 31.82, 'avg_duration': 18.145999999999997}]

In [70]:
new_df = spark.createDataFrame(data1)

new_df.show()

+------------------+------------+
|      avg_duration|max_duration|
+------------------+------------+
|18.145999999999997|       31.82|
+------------------+------------+



In [73]:
resp= df.join(new_df,df.PluginDuration == new_df.max_duration).\
select(FS.col("CPID").alias("chargepoint_id"),"max_duration","avg_duration")

In [78]:
resp.write.parquet("E:/workforce/spark/data/hudsonu-product/t1/analysis")


In [53]:
df.groupBy("CPID").agg(FS.max("PluginDuration").alias("max_duration"),FS.avg("PluginDuration").alias("avg_duration")).show()

+-------+------------+------------+
|   CPID|max_duration|avg_duration|
+-------+------------+------------+
|AN10218|       22.58|       22.58|
|AN22594|       20.88|       20.88|
|AN02137|       31.82|       31.82|
|AN15092|       11.82|       11.82|
|AN07263|        3.63|        3.63|
+-------+------------+------------+



In [25]:
transformed_df = df.withColumn("start_datatime", FS.to_timestamp(FS.expr("CONCAT(StartDate,' ',StartTime)"),'dd-MM-yyyy HH:mm:ss')).\
withColumn("end_datatime", FS.to_timestamp(FS.expr("CONCAT(EndDate,' ',EndTime)"),'dd-MM-yyyy HH:mm:ss')).\
select("CPID",FS.col("start_datatime",FS.col("end_datatime")))
transformed_df.drop("ChargingEvent","EndDate",)



+-------+-------------+----------+--------+------+--------------+----------+---------+-------------------+-------------------+
|   CPID|ChargingEvent|   EndDate| EndTime|Energy|PluginDuration| StartDate|StartTime|     start_datatime|       end_datatime|
+-------+-------------+----------+--------+------+--------------+----------+---------+-------------------+-------------------+
|AN22594|      2344473|03-06-2017|13:03:21|  11.5|   20.88388889|02-06-2017| 16:10:19|2017-06-02 16:10:19|2017-06-03 13:03:21|
|AN10218|     12184545|21-03-2017|20:18:29|  12.1|   22.58111111|20-03-2017| 21:43:37|2017-03-20 21:43:37|2017-03-21 20:18:29|
|AN02137|     11984777|08-03-2017|18:10:15|   7.8|   31.81611111|07-03-2017| 10:21:17|2017-03-07 10:21:17|2017-03-08 18:10:15|
|AN07263|     15554472|29-10-2017|17:08:00|   5.3|   3.633333333|29-10-2017| 13:30:00|2017-10-29 13:30:00|2017-10-29 17:08:00|
|AN15092|     15329256|15-10-2017|05:26:00|  19.2|   11.81666667|14-10-2017| 17:37:00|2017-10-14 17:37:00|2017-

In [32]:
filterred_data = transformed_df.withColumn('DiffInSeconds',FS.col("end_datatime").cast("long") - FS.col('start_datatime').cast("long")).\
select("CPID","start_datatime","end_datatime",FS.col("DiffInSeconds"))

In [35]:
fm = filterred_data.withColumn("diff_in_hour",FS.col("DiffInSeconds")/3600)

In [36]:
fm.show()

+-------+-------------------+-------------------+-------------+------------------+
|   CPID|     start_datatime|       end_datatime|DiffInSeconds|      diff_in_hour|
+-------+-------------------+-------------------+-------------+------------------+
|AN22594|2017-06-02 16:10:19|2017-06-03 13:03:21|        75182| 20.88388888888889|
|AN10218|2017-03-20 21:43:37|2017-03-21 20:18:29|        81292| 22.58111111111111|
|AN02137|2017-03-07 10:21:17|2017-03-08 18:10:15|       114538|31.816111111111113|
|AN07263|2017-10-29 13:30:00|2017-10-29 17:08:00|        13080|3.6333333333333333|
|AN15092|2017-10-14 17:37:00|2017-10-15 05:26:00|        42540|11.816666666666666|
+-------+-------------------+-------------------+-------------+------------------+



In [39]:
fm.groupBy("CPID").agg(FS.max("diff_in_hour").alias("max_duration"),FS.avg("diff_in_hour").alias("avg_duration")).show()

+-------+------------------+------------------+
|   CPID|      max_duration|      avg_duration|
+-------+------------------+------------------+
|AN10218| 22.58111111111111| 22.58111111111111|
|AN22594| 20.88388888888889| 20.88388888888889|
|AN02137|31.816111111111113|31.816111111111113|
|AN15092|11.816666666666666|11.816666666666666|
|AN07263|3.6333333333333333|3.6333333333333333|
+-------+------------------+------------------+

